# Checking enviroment states and action sets

In [6]:
import sys
import os.path
sys.path.append("..")
from SAC_Agent import *
from ENV_DETAILS import *
import os
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64"

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13718727980447188959
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22089629696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4588130141601417802
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-12-15 17:07:31.219220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-15 17:07:31.219340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-15 17:07:31.219385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-15 17:07:31.219464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-15 17:07:31.219509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

In [10]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_sac/' # Linux

ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [11]:

env = gym.make(ENV)
env

Exception: 
Missing path to your environment variable. 
Current values LD_LIBRARY_PATH=/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/local/cuda/lib64
Please add following line to .bashrc:
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/n/.mujoco/mujoco210/bin

In [5]:

max_steps = env._max_episode_steps
max_steps

NameError: name 'env' is not defined

In [ ]:
s = env.observation_space.sample()
s

In [ ]:
env.observation_space

In [ ]:
env.observation_space.shape

In [ ]:
s = env.reset()
s[0]

In [ ]:
s = env.reset()[0]
env.action_space, env.action_space, env.step(s)

In [ ]:
a = env.action_space.sample()
a, a.shape, a.reshape((1, len(a))).shape

In [ ]:
env.step(a)

In [ ]:
env.action_space.sample()

### Training/Hyperparam run

In [ ]:
%tensorboard --logdir logs_general/hyper

In [ ]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_sac/"


In [ ]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_sac"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/sac/",
                        end_of_episode = EPISODES, evaluation_epoch = env._max_episode_steps, training_steps = 500000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.95, discount_max = 0.99,
                lr_actor_min = 0.000001, lr_actor_max = 0.005,
                lr_critic_1_min = 0.000001, lr_critic_1_max = 0.005,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                tau_min = 0.001, tau_max = 0.1,
                reward_scaler = 1, num_layers_act = 4, num_layers_crit =4),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_sac/"

        training_steps = 1000000
        learning_rate= 0.001
        entropy_factor = 0.1
        discount = 0.99
        dense_units_actor = [128,64]
        num_layers_actor = 2
        dense_units_critic = [128,64]
        num_layers_crit = 2
        time_to_update= 100
        end_of_episode = 300
        tau = 0.01
        
        run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, end_of_episode,
                      environment_name = ENV,reward_scaler = 10, return_agent = False,lr_actor= 0.001, lr_critic_1= 0.001, lr_alpha = 0.001, tau = 0.001)
        

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:

val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_sac"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
training_steps = 2000000
discount  = best_hp["discount"]
lr_actor  = best_hp["lr_actor"]
lr_critic_1  = best_hp["lr_critic_1"]
tau  = best_hp["tau"]

end_of_episode = EPISODES
sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH
sucess_criteria_value = SUCESS_CRITERIA_VALUE
save_factor=50000
return_agent = True
reward_scaler = 100
environment_name=ENV

n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
dense_layers_actor = []
for i in range(n_dense_layers_actor):
    dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
dense_layers_critic = []
for i in range(n_dense_layers_critic):
    dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

env_model = run_training(training_steps = training_steps,
                         discount = discount,  
                         dense_units_act = dense_layers_actor,  
                         dense_units_crit = dense_layers_critic, 
                         num_layer_a = n_dense_layers_actor, 
                         num_layer_c = n_dense_layers_critic, 
                         writer = writer, 
                         end_of_episode = end_of_episode, 
                         save_factor = save_factor, 
                         sucess_criteria_epochs = sucess_criteria_epochs , 
                         sucess_criteria_value = sucess_criteria_value, 
                         environment_name = environment_name, 
                         reward_scaler = reward_scaler , 
                         return_agent = return_agent,
                         lr_actor = lr_actor, 
                         lr_critic_1 = lr_critic_1, 
                         tau = tau )

In [ ]:
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
